# Current

* Fix the hack (in particular remove the check for `isinstance(elem, CubeHermite)` in `representation.py` and generalise to 3D, etc.
* Dec 2016: I don't understand why there is no problem with the Argyris element any more since I pulled the latest changes from branch `next`. Before I had the same issues as with the CubicHermite...
* <strike>Implement (nodal) interpolation for Hermite: how should one compute the derivatives of the function to be interpolated?</strike> I'm using lambdas and autograd for now.
* <strike>Implement the trafo for the derivatives.</strike>
* <strike>Fix the issue with the mappings in the quadratures.</strike>
* <strike>Fix `_tabulate_dofs()` in `representation.py` to add the coordinates where dofs associated to derivative evaluations are .</strike>

# Some nomenclature

**Finite element** $(K,P,N)$: $K$ is the geometry, $P$ some polynomial space over $K$ and $N \subset P'$ a set of functions in the dual space of $P$ (the _degrees of freedom_) such that its dual $B \subset P$, i.e. a set of polynomials $p_i \in P$ such that $L_i(p_j) = \delta_{i j}$ for all $L_i \in N$, is a basis for all of $P$.

**dof** in ffc: element of the dual space $N=P'$. From the docs: 
>Each dof $L$ [in an element] is assumed to act on a field $f$ in the following manner: $$L(f) = w_{j, k} f_k(x_j),$$ where $w$ is a set of weights, $j$ is an index set corresponding to the number of points involved in the evaluation of the functional, and $k$ is a multi-index set with rank corresponding to the value rank of the function $f$. For common degrees of freedom such as point evaluations and directional component evaluations, there is just one point. However, for various integral moments, the integrals are evaluated using quadrature. The number of points therefore correspond[s] to the quadrature points. The points $x_j$, weights $w_{j, k}$ and components $k$ are extracted from FIAT (`functional.pt_dict`) in the intermediate representation stage.

By extension, **dof** is also used for the coefficients of the expansion of any function $f \in V_h$, where $V_h$ is the finite dimensional space built from all finite elements.

**Intermediate representation**: A dictionary built in `ffc/representation.py` for later code generation.

# Related commits and PRs

* [This commit](https://bitbucket.org/fenics-project/ffc/commits/8c96e6646aa7c42365a954e79d5df7ec4066b1e2) implements the "double covariant" transform in ffc. Modifies:
  * `ffc/evaluatebasis.py`
  * `ffc/evaluatebasisderivatives.py`
  * `ffc/evaluatedof.py`
  * `ffc/interpolatevertexvalues.py`
  * `ffc/quadrature/quadraturetransformer.py`
  * `uflacs/backends/ufc/evaluatebasis.py`

* [This PR](https://bitbucket.org/fenics-project/ufl/pull-requests/55/support-for-hellan-herrmann-johnson/diff) adds support for an element in UFL using the double covariant trafo. Modifies:
  * `ufl/algorithms/apply_function_pullbacks.py`
  * `ufl/finiteelement/elementlist.py`

# Intro

This notebook collects multiple tests done during the implementation of Hermite elements in FIAT and FFC. See:

* [hermite.tm](hermite.tm) for some of the math involved. 
* [quadrature.ipynb](quadrature.ipynb) for the implementation of the quadrature representation of forms requiring Hermite transforms.
* [boundary.ipynb](boundary.ipynb) for details and problems related to the specification of boundary conditions.
* ...

# Setup

In [ ]:
from dolfin import *
%matplotlib inline
import ufl
import ffc
import utils
import nbimporter
from shape_functions import hermite_shapes

import numpy as np
import matplotlib.pyplot as pl

# Force quadrature representation (we only implement this for now)
parameters["form_compiler"]["representation"] = "quadrature"

# Generate code for evaluate_basis_derivatives for testing
parameters["form_compiler"]["no-evaluate_basis_derivatives"] = False

TEST_DIMENSION = 1

def __nbinit__():
    global __all__
    __all__ = ['evaluate_shape_functions_mesh',
               'evaluate_shape_functions_reference',
               'compile_element']

In [ ]:
ffc.log.set_level(ffc.log.DEBUG)
#ffc.log.add_logfile("/tmp/fenics.log")
#ffc.__version__, ffc.supported_elements

In [ ]:
if TEST_DIMENSION == 1:
    mesh = UnitIntervalMesh(10)
elif TEST_DIMENSION == 2:
    mesh = UnitSquareMesh(10, 10)
elif TEST_DIMENSION == 3:
    mesh = UnitCubeMesh(10, 10, 10)

V = FunctionSpace(mesh, "Hermite", 3)

After a quick hack we were able to create the Hermite UFL element with

```python
elem = ufl.FiniteElement('Hermite', mesh.ufl_cell(), 3)
```

We did this first to find out what happens at the FFC level when we try to build a `FunctionSpace`. 

**FIXME** We need to specify the degree even though only CubicHermite elements are supported as of yet.

Creating a `FunctionSpace` with Hermite elements involves (at least):

1. We must "register" the Hermite element in `ufl/finitelements/elementlist.py` in order to be able to use the string argument "Hermite".

2. For each basis element of the element which is a derivative (i.e. two at every vertex of the triangle in 2D) we collect in the *intermediate representation* (`ir`) the coefficients of *all* basis elements which are derivatives (i.e. two at every vertex of the triangle in 2D). This is done in `ffc/representation.py`, `_evaluatebasis()` by adding the coefficients to `dof_data`.

3. Inside `ffc/evaluatebasis.py` we use these coefficients and the Jacobian to apply the transformation in `_compute_values()`, inside a new case for mapping "hermite". **FIXME:** Maybe I should change the Hermite mapping to act on point evaluations (as the identity) and directional derivatives (as a linear combination using the domain transformation) in order to have a more "homogeneous" implementation (i.e. without as many cases, etc.). That is: reuse the idea of the matrix $H$ as in the quadratures.

4. Interpolation cannot work without computing derivatives of arbitrary `ufc::function`s, see below.

5. Quadratures also must feature the Hermite transform. A naive approach yields unwieldy expressions with hundreds of terms for even the simplest of forms.

6. ...

# First tests (finish)

A minor sanity check for the degrees:

In [ ]:
import FIAT

ufl_element = ufl.FiniteElement('Hermite', mesh.ufl_cell(), 3)
fiat_element = ffc.fiatinterface._create_fiat_element(ufl_element)

print(ufl_element.degree(), fiat_element.degree(), isinstance(fiat_element, FIAT.hermite.CubicHermite))

Now check the types of mappings for each dof in the element:

In [ ]:
print(fiat_element.mapping())

**NOTE:** Maybe I should use the same idea as I did in the quadratures of collecting both affine and hermite trafos into one $(d+1) \mathrm{x} (d+1)$ matrix

# FFC  element compilation

In [ ]:
def compile_element(element, destdir='/tmp', params=None):
    """ Compiles elements with FFC.
    Uses quadrature representation by default.
    
    Arguments:
    ----------
        element: a UFL FiniteElement
        destdir: where to save the C header file.
        params: dictionary of options to pass to FFC. Will be used
                to update() the defaults:
                        {'no-evaluate_basis_derivatives': False, 
                         'representation': 'quadrature', 
                         'quadrature_degree': 5}
    
    Returns:
    --------
        Complete path to the output file.
    """
    from ffc import compile_element
    from os import path
    from ufl import FiniteElement, FiniteElementBase
    if not isinstance(element, FiniteElementBase):
        raise ValueError("type(element) = %s does not derive from ufl.FiniteElementBase" %
                         type(element))

    family = element.family()
    tdim = element.cell().topological_dimension()
    fc_params = {'no-evaluate_basis_derivatives': False, 
                 'representation': 'quadrature', 
                 'quadrature_degree': 5}   # FIXME: what's the right number here
    if params is not None:
        fc_params.update(params)
    output_fname = path.join(destdir, "%s-%dD.h" % (family, tdim))
    with open(output_fname, 'wt') as fd:
        out = compile_element(element, 
                              prefix=family[:3].lower(),
                              parameters=fc_params)
        fd.write(out[0])

        return output_fname

In [ ]:
from dolfin import FiniteElement, MixedElement
compile_element(FiniteElement('Hermite', 'interval'))
compile_element(FiniteElement('Hermite', 'triangle'))
compile_element(FiniteElement('Hermite', 'tetrahedron'))

## Mixed elements

In [ ]:
compile_element(MixedElement([FiniteElement("Lagrange", "interval", 2),
                              FiniteElement("Hermite", "interval", 3)]))

# Checking FFC's shape functions

We pick a simple 1D grid and manually apply the necessary transformation to the basis functions computed above in ["Hermite shape functions in 1D"](#Hermite-shape-functions-in-1D) in order to compare them to the ones compiled by FFC. First we write a simple function to evaluate shape functions and their derivatives.

In [ ]:
def evaluate_shape_functions_mesh(V, points, num_derivatives=1):
    """ Evaluates shape functions and their derivatives for scalar
    functions defined over 1D meshes.

    Arguments:
    ----------
        V: FunctionSpace
        points: (n,) array of *physical* coordinates.
        
    Returns:
    --------
        A tuple (values, derivs), where both are np.ndarrays, of
        shape = (V.element().space_dimension(), len(points)).
    """
    e = V.element()
    m = V.mesh()
    dim = e.space_dimension()
    val = np.zeros(dim)
    der = np.zeros((dim, num_derivatives))
    values = np.zeros((dim, points.size))
    derivs = np.zeros((dim, num_derivatives, points.size))
    for j, x in enumerate(points):
        x = np.array([x])
        cell_id = m.bounding_box_tree().compute_first_entity_collision(Point(x))
        cell = Cell(m, cell_id)
        coordinate_dofs = cell.get_coordinate_dofs()
        e.evaluate_basis_all(val, x, coordinate_dofs, cell.orientation())
        values[:,j] = val.copy()
        e.evaluate_basis_derivatives_all(num_derivatives, der, x,
                                         coordinate_dofs, cell.orientation())
        derivs[:,:,j] = der.copy()

    return values, derivs

In [ ]:
def evaluate_shape_functions_reference(element, points, num_derivatives=1):
    """ Evaluates shape functions and their derivatives over points in the
    reference 1D simplex.

    Arguments:
    ----------
        element: dolfin.cpp.fem.FiniteElement 
        points: (n,) array of *reference* coordinates (i.e. in [0,1])
        num_derivatives: Number of derivatives to take.
        
    Returns:
    --------
        A tuple (values, derivs), where both are np.ndarrays, and
        values.shape = (element.space_dimension(), points.size).
        derivs.shape = (element.space_dimension(), num_derivatives, len(points)).
    """
    assert 0. <= points.min() and points.max() <= 1., \
           "Points for evaluation must lie in the reference interval"

    if num_derivatives > 1:
        print("CAREFUL: what does the array of derivatives mean??")

    coordinate_dofs = np.array([0., 1.]) # HACK: there has to be a better way
    dim = element.space_dimension()
    val = np.zeros(dim)
    der = np.zeros((dim, num_derivatives))
    values = np.zeros((dim, points.size))
    derivs = np.zeros((dim, num_derivatives, points.size))
    for j, x in enumerate(points):
        x = np.array([x])
        element.evaluate_basis_all(val, x, coordinate_dofs, 0)
        values[:,j] = val.copy()
        element.evaluate_basis_derivatives_all(num_derivatives, der, x,
                                               coordinate_dofs, 0)
        derivs[:,:,j] = der.copy()

    return values, derivs

In [ ]:
vals, ders = evaluate_shape_functions_reference(V.element(), np.array([0., 0.5, 1.]), 2)

ders[0].round(3)

Recall that the transformation for Hermite shape functions is given by:

$$ \phi_i(x) = H_i \hat{\phi}_i(\hat{x}(x)),\ i \in \{0,1,2,3\}$$

where $H = (1,J,1,J)$ because the functions associated to partial derivatives at the nodes are at indices $1,3$. Therefore the derivatives are given by

$$ \phi_i' (x) = H_i  \hat{\phi}_i' (\hat{x} (x)) K = \left\{
   \begin{array}{ll}
     \hat{\phi}_i' (\hat{x} (x)), & i \in \{ 1, 3 \},\\
     K \hat{\phi}_i' (\hat{x} (x)), & i \in \{ 0, 2 \} .
   \end{array} \right. $$

In [ ]:
import autograd as ad
V = FunctionSpace(UnitIntervalMesh(2), "Hermite", 3)
sh = hermite_shapes(1)
shd = [ad.elementwise_grad(f, 0) for f in sh]
xx = np.linspace(0, 0.5, 100)
values, derivs = evaluate_shape_functions_mesh(V, xx, 1)

def J(x):
    """ Direct trafo / Jacobian for [0,0.5] to [0,1]"""
    return 0.5*x
def K(x):
    """ Inverse trafo / Inverse of Jacobian for [0,0.5] to [0,1]"""
    return 2.0*x
def identity(x):
    return x

trafo = [J if i in [1,3] else identity for i in range(4)]
pl.figure(figsize=(10,4))
pl.subplot(1,2,1)
for i in range(4): #[1,3]:
    pl.plot(xx, values[i], label="$\phi_{%d}$" % i)
    if not np.allclose(values[i], trafo[i](sh[i](K(xx)))):
        print("Shape functions %d differ" % i)
pl.title("Shape functions")
pl.legend()
pl.subplot(1,2,2)
trafo = [K if i in [0,2] else identity for i in range(4)]
for i in range(4): #[1,3]:
    yy = derivs[i][0]
    dd = trafo[i](shd[i](K(xx)))
    pl.plot(xx, yy, label="$\phi_{%d}'$" % i)
    if not np.allclose(yy, dd):
        print("Derivatives of shape functions %d differ by %f" % 
              (i, np.linalg.norm(yy - dd, np.inf)))
pl.title("Derivatives of the shape functions")
_ = pl.legend()

In [ ]:
mesh = UnitIntervalMesh(2)
V = FunctionSpace(mesh, "Lagrange", 2)
f = Function(V)
fv = f.vector()
fv.set_local(np.array([0,0,0,0,1], dtype=np.float))

## CAREFUL!

dolfin's plot() uses `f.compute_vertex_values()` on the mesh to plot, instead of `f()`
See:
```python
import inspect
inspect.findsource(plot)
```
**`f.compute_vertex_values()` calls `element.interpolate_vertex_values()` internally.**

But this is not an issue. The code *generating* the method:

```c++
void interpolate_vertex_values(double * vertex_values,
                                 const double * dof_values,
                                 const double * coordinate_dofs,
                                 int cell_orientation,
                                 const ufc::cell& c) const final override
```

uses the nodal property of PointEvaluation basis functions to do roughly the following:

1. Compute values of the $N$ basis functions at all the $M$ vertices of the cell. (I guess these agree with the `coordinate_dofs`, but this argument is not used in the code generated for Lagrange nodal basis functions, probably optimised away because of the "delta property")
2. Generate the scalar product of `dof_values` (coefficients of some discrete function $f_h$ in the basis of $V_h$) with each of the $M$ arrays computed above (i.e. for each vertex). Any terms close to zero are removed from the generated code. In the Hermite case, all basis functions coming from PointDerivatives are zero at the vertices, so the corresponding `dof_values` are discarded and all that is left is `dof_values[i] * 1.0`.
3. Assign the result of this scalar product to  `vertex_value[i]`.

In [ ]:
xx = np.linspace(0,1,100)
pl.figure(figsize=(12,8))
pl.subplot(2,2,1)
plot(f, title="f, interpolate_vertex_values")
pl.ylim((-1,1))
pl.subplot(2,2,3)
plot(f.dx(0), title="f.dx(0), interpolate_vertex_values")
pl.ylim((-3,1))
pl.subplot(2,2,2)
pl.plot(xx, [f(x) for x in xx])
pl.ylim((-1,1))
pl.title("f, evaluated at pts")
pl.subplot(2,2,4)
pl.plot(xx, [f.dx()(x) for x in xx])
pl.title("f.dx(), evaluated at pts")
_ = pl.ylim((-1,1))

In [ ]:
#inspect.getfile(function.Lagrange)
#Dx(f,0)(1)  # evaluation of derivatives not implemented in fenics yet

In [ ]:
f.vector().array(), f.dx().vector().array()

In [ ]:
f.compute_vertex_values(V.mesh())

# Basis evaluation at mesh nodes

Here we test that the basis functions of $V$ fulfill the "delta property"

$$ \partial_j \phi_{\alpha_i} ( v_{\beta} ) = \delta_{\alpha \beta} \delta_{i j}, $$

where $i, j \in \{0,1,...,d\}, \partial_0 \ f = f$, and $\alpha, \beta$ run over the set of indices of vertices of the simplex in $\mathbb{R}^d$.

In [ ]:
def evaluate_basis_at_points(V, points):
    """ A quick hack to test the delta property on mesh nodes.

    Arguments:
    ----------
        V: FunctionSpace
        xx: np.array of points
    """
    def highlight(s, num):
        trafo = utils.yellow if np.isclose(num, 1.) else lambda x:x
        print(trafo(s + "%.2f" % np.round(num, 2)))
        
    element = V.element()
    mesh = V.mesh()
    # Array to store the values
    valdim = element.value_dimension(0)
    assert valdim == 1, "Real valued shape functions expected."
    tdim = element.topological_dimension()
    values = np.zeros(valdim, dtype=np.double)
    derivs = np.zeros(valdim * tdim, dtype=np.double)
    print("DOF, [derivative] @ [x, y] = value   (only if any of DOF or derivatives != 0)\n")
    for xyz in points:
        xyz = np.array(xyz, dtype=np.double)
        pt = Point(xyz) 
        cell_id = mesh.bounding_box_tree().compute_first_entity_collision(pt)
        cell = Cell(mesh, cell_id)
        coordinate_dofs = cell.get_coordinate_dofs()

        print("***************************")
        for i in range(element.space_dimension()): 
            element.evaluate_basis(i, values, xyz, coordinate_dofs, cell.orientation())
            element.evaluate_basis_derivatives(i, 1, derivs, xyz, 
                                               coordinate_dofs, cell.orientation())
            if not np.isclose(values, 0.) or not np.allclose(derivs, 0.):
                highlight("%d   @ %s = " % (i, xyz), values)
                #else:
                for j in range(tdim):
                    highlight(" ,%d @ %s = " % (j, xyz), derivs[j])

In [ ]:
evaluate_basis_at_points(V, V.mesh().coordinates())

In [ ]:
_ = plot(V.mesh())

# Projection

Simple projection of a constant function already shows some weird stuff. I think `project()` is using the right norm (it should fetch it from FIAT's declaration that HER3 is in $H^1$), but maybe there's more to it.

In [ ]:
f = Constant(2.0)
u = project(f, V)
print("f is a %s and u is a %s" % (f.ufl_element(), u.ufl_element()))
if TEST_DIMENSION == 1: # plot() needs dim > 1
    xx = V.mesh().coordinates()
    _ = pl.plot(xx, [u(x) for x in xx])
else:
    _ = plot(u, cmap='bone')

Note how the projection oscillates when we are not at grid vertices. However total error seems to be within reasonable bounds for double precision ($10^{-13}$)

In [ ]:
if V.element().topological_dimension() == 2:  # This test only works in 2D
    xy = V.mesh().coordinates()
    grid_xx = xy[:11,0]
    xx = np.arange(0, 1, 0.01)
    y = 0.97
    pl.plot(xx, [u(x, y) for x in xx])
    pl.scatter(grid_xx, [u(x, y) for x in grid_xx], c='r')
    _ = pl.xlim(-0.01,1.01)
    
    vals = u.compute_vertex_values(V.mesh())[:, None]
    print("Total error: %.3g" % np.linalg.norm(2. - vals))

Here are a couple more examples. Notice that we first declare the function to be projected onto $V$ as living in another finite element space.

In [ ]:
tdim = V.element().topological_dimension()
if tdim == 1:
    f = Expression("sqrt(x[0])*x[0]", degree=3)
    u = project(f, V)
    vals = u.compute_vertex_values(V.mesh())[:, None]
    coordinates = V.mesh().coordinates()
    errors = vals - np.sqrt(coordinates)*coordinates
    _ = pl.plot(coordinates, errors)
elif tdim == 2:
    f = Expression("x[0]*x[1]", degree=3)
    u = project(f, V)
    vals = u.compute_vertex_values(V.mesh())
    coordinates = V.mesh().coordinates()
    errors = vals - coordinates.T[0] * coordinates.T[1]
    plot(u, cmap='bone')
    pl.scatter(coordinates.T[0], coordinates.T[1], c=errors, s=20, cmap='hot', linewidths=0)
    pl.axes().set_aspect('equal')
print("f is a %s and u is a %s" % (f.ufl_element(), u.ufl_element()))
print("Total error: %.3g" % np.linalg.norm(errors))

In [ ]:
_ = pl.plot(errors)

Here's another example, with a derivative. Notice that we need to explicitly declare the domain for $f$ or Expression won't be able to infer its dimension.

In [ ]:
if TEST_DIMENSION == 2:
    f = Expression("sqrt((x[0]-0.5)*(x[0]-0.5)+(x[1]-0.5)*(x[1]-0.5))", degree=3, domain=u.ufl_domain())
    df = f.dx()
    u = project(df, V)
    _ = plot(u, cmap='bone')
    print("df is a %s and u is a %s" % (df.ufl_element(), u.ufl_element()))

A final example with Hermite shape functions (**FIXME** are these even right?)

In [ ]:
# These three are for the triangle {(-1,-1), (1,-1), (-1,1)}
phi1 = Expression("(7/8)*x[0] + (7/8)*x[1] + (13/8)*x[0]*x[0] + (13/4)*x[0]*x[1]"
                  "+ (13/8)*x[1]*x[1] + (1/4)*x[0]*x[0]*x[0] + (13/8)*x[0]*x[0]*x[1]"
                  "+ (13/8)*x[0]*x[1]*x[1] + (1/4)*x[1]*x[1]*x[1]", degree=3)
phi2 = Expression("1/2 + (13/8)*x[0] + (7/8)*x[1] + (7/8)*x[0]*x[0] + (7/4)*x[0]*x[1]"
                  "+ (7/8)*x[1]*x[1] - (1/4)*x[0]*x[0]*x[0] + (7/8)*x[0]*x[0]*x[1]"
                  "+ (7/8)*x[0]*x[1]*x[1]", degree=3)
phi3 = Expression("1/2 + (7/8)*x[0] + (13/8)*x[1] + (7/8)*x[0]*x[0] + (7/4)*x[0]*x[1]"
                  "+ (7/8)*x[1]*x[1] + (7/8)*x[0]*x[0]*x[1] + (7/8)*x[0]*x[1]*x[1]"
                  "- (1/4)*x[1]*x[1]*x[1]",
                 degree=3)

#mesh = refine(refine(refine(refine(UnitTriangleMesh()))))
#elem = ufl.FiniteElement('Hermite', mesh.ufl_cell(), 3)
#W = FunctionSpace(mesh, elem)

v = project(phi1, V)
_ = plot(v, cmap='bone')

## Projecting from the same space

Up until now, we've declared $f$ to live in a different space, then projected. If we take $f \in V$, then a call to `project()` most likely tries to call `interpolate()` instead. This calls `create_function()` at some point, which requires handling of different transformations. After brute-hacking that to accept Hermite trafos, we hit the problem that `evaluate_dof()` (i.e. interpolation) cannot be implemented for Hermite elements because we are not handling the derivatives (yet), so I disabled it by throwing an exception:

In [ ]:
f = Expression("x[0]*x[0] + x[1]*x[1]", element=V.ufl_element())
# f.ufl_element() is FiniteElement('Hermite', triangle, 3)
u = project(f, V)  # throws exception
_ = plot(u, cmap='bone')
print("f is a %s and u is a %s" % (f.ufl_element(), u.ufl_element()))

# Comparison to Lagrange elements

We project the same function onto P3:

In [ ]:
W = FunctionSpace(V.mesh(), 'P', 3)
f = Expression("x[0]*x[0] + x[1]*x[1]", degree=3)
u = project(f, W)
_ = plot(u, cmap='bone')

Projection of a constant looks much better than with Hermite elements:

In [ ]:
u = project(Constant(2.0), W)
_ = plot(u, cmap='bone')

# Extracting the nodal values

See section 5.2.6 of the tutorial.

In [ ]:
f = Expression("x[0]*x[0] + x[1]*x[1]", degree=3)
u = project(f, V)

vals = u.compute_vertex_values(V.mesh())
coordinates = V.mesh().coordinates()

import matplotlib.pyplot as pl
pl.scatter(coordinates.T[0], coordinates.T[1], c=vals, s=100, cmap='bone', linewidths=0)
_ = pl.axes().set_aspect('equal')

# Old things

## Test construction of polynomial set

This is taken from the constructor of `FIAT.finite_element.FiniteElement()`.

**TODO:** I'd like to test whether the coefficients for the nodal basis functions agree with what I should be getting. 

In [ ]:
# build generalized Vandermonde matrix
from FIAT import polynomial_set, reference_element, hermite
K = reference_element.ReferenceElement(shape = reference_element.LINE,
                                       vertices = ((-1.0,),(1.0,)),
                                       topology = reference_element.UFCInterval().get_topology())
poly_set = polynomial_set.ONPolynomialSet(K, 3)
old_coeffs = poly_set.get_coeffs()
dual = hermite.CubicHermiteDualSet(K)
dualmat = dual.to_riesz(poly_set)

shp = dualmat.shape
if len(shp) > 2:
    num_cols = np.prod(shp[1:])

    A = np.reshape(dualmat, (dualmat.shape[0], num_cols))
    B = np.reshape(old_coeffs, (old_coeffs.shape[0], num_cols))
else:
    A = dualmat
    B = old_coeffs

V = np.dot(A, np.transpose(B))

Vinv = np.linalg.inv(V)

new_coeffs_flat = np.dot(np.transpose(Vinv), B)

new_shp = tuple([new_coeffs_flat.shape[0]] + list(shp[1:]))
new_coeffs = np.reshape(new_coeffs_flat, new_shp)

#poly_set = PolynomialSet(K, poly_set.get_degree(), poly_set.get_embedded_degree(),
#                         poly_set.get_expansion_set(), new_coeffs, poly_set.get_dmats())

In [ ]:
H = hermite.CubicHermite(K)
np.round(H.poly_set.tabulate_new(K.get_vertices()),2)

## Test transformation of basis functions

This is copied from `transform_hermite.py`, in a previous release of FIAT. It tests whether the nodal basis evaluated on a lattice over the reference element and over a transformed  element transforms well under an affine mapping.

Basically, what we do is we compute the "hermite" transformation in `evaluatebasis.py` by hand:

1. Tabulate (compute) the values of each basis function of the reference element over a lattice of points in the triangle, then multiply it by an adequate transformation matrix (mixing the values for the nodal basis functions related to derivatives).
2. Tabulate the values of each basis function defined over a different simplex.
3. Compare both results

In [ ]:
from FIAT import reference_element, hermite

# Let's set up the reference triangle and another one
Khat = reference_element.UFCTriangle()
K = reference_element.ReferenceElement(shape = reference_element.TRIANGLE,
                                       vertices = ((-1.0,-1.0),(1.0,-1.0),(-1.0,1.0)),
                                       topology = Khat.get_topology())

# Construct the affine mapping between them
A, b = reference_element.make_affine_mapping(K.get_vertices(), Khat.get_vertices())

# build the Hermite element on the two triangles
Hhat = hermite.CubicHermite(Khat)
H = hermite.CubicHermite(K)

# get some points on each triangle
pts_hat = Khat.make_lattice(6)
pts = K.make_lattice(6)

# as a sanity check on the affine mapping, make sure pts map to pts_hat
for i, p in enumerate(pts):
    if not np.allclose(pts_hat[i], np.dot(A, p) + b):
        print("barf")

In [ ]:
# Tabulate the Hermite basis on each triangle
# Each column is the value of one nodal basis function evaluated at every point
Hhat_tabulated = Hhat.get_nodal_basis().tabulate_new(pts_hat)
H_tabulated = H.get_nodal_basis().tabulate_new(pts)

### Prepare global transformation matrix
M = np.zeros((10,10), dtype=np.double)
Ainv = np.linalg.inv(A)

# Point values are transformed as is
# Derivative values are "mixed" with the Jacobian.
M[0,0] = 1.0
M[3,3] = 1.0
M[6,6] = 1.0
M[9,9] = 1.0
M[1:3,1:3] = np.transpose(Ainv)
M[4:6,4:6] = np.transpose(Ainv)
M[7:9,7:9] = np.transpose(Ainv)

print(np.allclose(H_tabulated, np.dot(M.T, Hhat_tabulated)))

In [ ]:
Hhat_tabulated.shape

In [ ]:
basis = H.get_nodal_basis()
np.round(basis.tabulate_new([(-1,-1),(-1,0),(0,0)]))

## What are dmats ("expansion coefficients for basis function derivative") ?

In [ ]:
from FIAT import lagrange
Hhat = lagrange.Lagrange(Khat, 1)
basis = Hhat.get_nodal_basis()
dmats = basis.get_dmats()
print(len(dmats), dmats[0].shape, dmats[1].shape)
print(np.round(dmats[0], 1))
print(np.round(dmats[1], 1))